In [7]:
import csv
import random
import concurrent.futures

from itertools import *
from functools import *
from typing import *

import numpy as np

import scipy as sp
import scipy.sparse
import scipy.sparse.linalg

from overloading import overload
from tqdm import tqdm

In [8]:
import numpy as np

def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [9]:
DATA_PREFIX = ''

# Read data

In [10]:
import os
os.chdir('{}/{}'.format(os.getcwd(),
                        DATA_PREFIX))

In [11]:
def get_likes() -> Callable[[], Tuple[str, str]]:
    """Return an iterator over user->item edges."""
    train_likes_file = open('train_likes.csv')
    reader = csv.DictReader(train_likes_file)
    return map(lambda row: (row['user_id'], row['item_id']), reader)

In [12]:
user_item_pairs = list(get_likes())

In [13]:
users, items = map(tuple, map(set, zip(*user_item_pairs)))

user_to_i = {user: i for i, user in enumerate(users)}
item_to_i = {item: i for i, item in enumerate(items)}

In [14]:
# We could construct a CSR matrix from the very start, but that's not very convenient for holdoff

#ix, iy = zip(* ((user_to_i[u],item_to_i[i]) for (u,i) in user_item_pairs) )
#values = np.ones_like(ix,dtype='bool')

#A = sp.sparse.csr_matrix((values,(ix,iy) ))

In [15]:
# Therefore, we construct a LIL matrix and, having populated it, convert to CSR

A = sp.sparse.lil_matrix((len(users), len(items))) # a matrix we'll run SVD on, with some connections taken out (held off)
A_full = sp.sparse.lil_matrix((len(users), len(items))) # a matrix we'll check against, with all connections in place

for user, item in user_item_pairs:
    A[user_to_i[user], item_to_i[item]] = (True if random.random() > 0.15 else False) # NB: non-determenistic!
    A_full[user_to_i[user], item_to_i[item]] = True
    
A = A.tocsr()
A_full = A_full.tocsr()

In [16]:
A

<55863x23891 sparse matrix of type '<class 'numpy.float64'>'
	with 94147 stored elements in Compressed Sparse Row format>

# SVD

$\DeclareMathOperator*{\argmin}{arg\,min}$
There are two different ways to look at the truncated SVD of a matrix. One is the standard definition:

> First you do the SVD: $\underset{n\times m}{X} = \underset{n\times n}{U} \overset{n\times m}{\Sigma} \underset{m\times m}{V^T}$, where $U$ and $V$ are rotation matrices, and $\Sigma$ has the singular values along the diagonal. Then you pick the top $k$ singular values, zero out the rest, and hack off irrelevant rows and columns to make a $k$-rank approximation to the original: $X \approx \tilde{X} = \underset{n\times k}{\tilde{U}} \overset{k\times k}{\tilde{\Sigma}} \underset{k\times m}{\tilde{V}^T}$

This is all fine and dandy, but it doesn't make sense when talking about matrices with missing values. However, there's an interesting property of the $k$-truncated SVD--It's the best $k$-rank approximation to the original! That is:

> $ \tilde{X} = \argmin_{B : rank(B)=k} \displaystyle\sum\limits_{i,j} (X_{ij} - B_{ij})^2$

This property seems easy to generalize to the missing value case. Basically you're looking for a $k$-rank matrix that minimizes the element-wise mean squared error across the known entries of the original matrix. That is, when you're training the system, you ignore all of the missing values.

Then, once you've come up with a suitably "close" $k$-rank approximation to the original, you use it to fill in the missing values. That is, if $X_{ij}$ was missing, then you fill in $\tilde{X}_{ij}$. Tada! You are now done.

-- http://stats.stackexchange.com/a/35460

In [17]:
u, s, vt = sp.sparse.linalg.svds(A.astype(np.float32), k=100) # note the static components count. It could be optimised automatically via gradient descent on mAP.

In [18]:
u.shape, s.shape, vt.shape

((55863, 100), (100,), (100, 23891))

In [19]:
u

array([[  1.51224391e-04,   1.42343738e-03,  -9.54224379e-04, ...,
          4.31466469e-04,  -6.37803285e-04,   6.43226784e-04],
       [ -1.08867993e-04,  -7.53359869e-04,  -2.57339634e-05, ...,
          1.95169177e-05,  -4.19743119e-05,   3.92163674e-05],
       [  7.27701536e-06,   1.47315220e-03,   7.23484991e-05, ...,
          5.86809125e-03,  -2.26987083e-03,   2.85436958e-03],
       ..., 
       [  1.98873779e-04,  -5.58835200e-05,  -5.72291428e-05, ...,
         -4.85092896e-05,  -1.26747196e-04,   1.01339167e-04],
       [  1.05078134e-10,   4.05027401e-11,  -2.76464768e-10, ...,
         -8.30226513e-12,   3.82780717e-11,  -2.68037988e-11],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]], dtype=float32)

In [20]:
s

array([ 10.43182755,  10.44400883,  10.46144772,  10.50055122,
        10.53884506,  10.54823303,  10.58362675,  10.60163021,
        10.6730299 ,  10.71234226,  10.74433041,  10.8058176 ,
        10.82253456,  10.8674984 ,  10.92055511,  10.93682289,
        10.96995354,  11.01584435,  11.07764244,  11.10638142,
        11.13105774,  11.15678787,  11.19907951,  11.25809669,
        11.33368874,  11.3558588 ,  11.38833714,  11.43347359,
        11.44468307,  11.55458164,  11.60127544,  11.61484814,
        11.64925671,  11.71235085,  11.78900719,  11.83900833,
        11.87044716,  11.95395088,  11.99664783,  12.03398037,
        12.11687946,  12.16360664,  12.24505806,  12.25090599,
        12.28740597,  12.42333698,  12.53813362,  12.61589146,
        12.69969273,  12.72949505,  12.86244678,  12.87318993,
        12.92994785,  13.11477375,  13.28208733,  13.32517624,
        13.34020805,  13.53727531,  13.6368618 ,  13.65943909,
        13.85501575,  13.8755722 ,  13.94878292,  13.99

In [21]:
vt

array([[  1.70220039e-04,   6.51082781e-04,  -1.45500875e-03, ...,
         -3.57008050e-03,   2.04894074e-10,  -3.53676267e-04],
       [  2.09082529e-04,  -4.46078740e-03,  -4.36407002e-03, ...,
          4.31250595e-03,  -5.78737891e-11,  -3.75625041e-06],
       [  3.03882916e-06,   2.19747121e-03,  -8.99397768e-03, ...,
          1.14748394e-03,  -5.25458344e-10,  -8.34933162e-05],
       ..., 
       [  1.95548128e-05,  -3.10054515e-04,  -2.65162438e-04, ...,
          6.15155790e-04,   2.97958824e-10,   4.95865970e-06],
       [ -2.62769008e-05,  -7.26741040e-04,  -2.64114235e-03, ...,
         -1.01307430e-03,   4.49148052e-10,  -8.93385368e-06],
       [  2.77270155e-05,   5.81866130e-04,   2.21736054e-03, ...,
          9.75851901e-04,  -3.42192830e-10,   7.28612940e-06]], dtype=float32)

# Predicting

Let the matrix A be such that rows are the users and the columns are the items that the user likes. One way to think of SVD is as follows: SVD finds a hidden feature space where the users and items they like have feature vectors that are closely aligned.

So, when we compute $A = U \times s \times V^T$, the $U$ matrix represents the feature vectors corresponding to the users in the hidden feature space and the $V^T$ matrix represents the feature vectors corresponding to the items in the hidden feature space.

Now, if I give you two vectors from the same feature space and ask you to find if they are similar, what is the simplest thing that you can think of for accomplishing that? Dot product.

So, if I want to see user $i$ likes item $j$, all I need to do is take the dot product of the $i$th entry in $U$ and $j$th entry in $V^T$. Of course, dot product is by no means the only thing you can apply, any similarity measure that you can think of is applicable.

-- http://stats.stackexchange.com/a/31101

In [22]:
np.dot(u[:10,] * s, vt) # get the probabilities for first ten users -> all films

array([[  1.16081383e-05,  -3.87252599e-04,  -4.38264105e-04, ...,
         -2.45513103e-04,  -2.85116895e-12,  -2.98090072e-06],
       [  4.56361959e-05,   5.82863831e-05,   8.56930783e-05, ...,
         -2.00049762e-05,   1.77645884e-11,  -3.53352584e-06],
       [ -6.35085053e-06,  -4.21985373e-04,  -7.82952528e-04, ...,
         -2.69662181e-04,   9.66845493e-10,  -6.61512968e-06],
       ..., 
       [ -2.30886212e-06,  -2.18786245e-05,  -1.69816631e-05, ...,
         -3.56906712e-05,   2.73376374e-12,   3.44605894e-07],
       [ -8.34535513e-07,  -2.16327200e-04,   2.69119482e-04, ...,
          3.59944464e-03,  -2.84170847e-11,   4.41086877e-06],
       [  7.59548420e-05,   1.63610477e-03,   4.17427393e-03, ...,
         -1.86571432e-03,   3.44414983e-11,   1.69327031e-05]], dtype=float32)

In [23]:
np.dot(u[1,:] * s, vt[:,546]) # get the probabilities for user 1 -> film 546

-1.8618677e-05

### Recommended interface
```python
get_like_confidence(user_id: str, item_id: str) -> float
```

In [24]:
@overload
def get_like_confidence(user_id: int, item_id: int) -> float:
    return np.dot(u[user_id,:] * s, vt[:,item_id])

In [25]:
@overload
def get_like_confidence(user_id: str, item_id: str) -> float:
    return get_like_confidence(user_to_i[user_id],
                               item_to_i[item_id])

In [26]:
get_like_confidence('70a2805f307f49ec42d4309190daa587', '0ccd2d96eb4b393ecb8cf4e55a6544b6')

0.00033780001

In [27]:
for i, j in list(islice(get_likes(), 10)):
    x = get_like_confidence(i, j)
    print(x)

5.12165e-17
0.992627
7.29326e-06
0.128027
0.0
0.99172
0.992728
0.995031
3.2594e-10
1.00534


### Ranking interface
```python
def recommend(user_id: str, k: int=10) -> Sequence[str]
```

In [28]:
@overload
def recommend(user_id: int, k: int=10) -> Sequence[int]:
    
    result = np.dot(u[user_id,:] * s, vt).argsort()[::-1]

    if k is None:
        # do not truncate
        return result
    
    return result[:k]

In [29]:
@overload
def recommend(user_id: str, k: int=10) -> Sequence[str]:
    return list(map(lambda x: items[x], recommend(user_to_i[user_id], k)))

In [30]:
recommend(0)

array([23661, 23442,  3549, 14202, 21786, 20269,  3125,  8595, 19871,  8071])

In [31]:
recommend('70a2805f307f49ec42d4309190daa587', k=5)

['e1cdbda92a8167c8a6c994872fd32b3e',
 '8780d3bc34fcfccf5a76850bd58d3952',
 '7b098a5f5901ab25acd03bb602919032',
 'b513db7287f1b585a7cffc82251e0219',
 '8f224d9f39b532b174f0147b8bfc9aa3']

# Check

Here, we estimate quality of our predictions via mAP@k - a ranking metric (like NDCG) which values the order of elements in addition to their plain accountance in the recommendation.

**Note** that the models cannot be compared (a) with different `k` and (b) by results of one run due to the population process being non-determenistic.

In [32]:
def get_actual(user_id: int) -> Sequence[int]:
    """Returns items user actually liked (ground truth)."""
    return np.where(A_full[user_id, :].toarray()[0] == 1)[0]

In [33]:
get_actual(4214)

array([2328])

In [34]:
# AP for user 0

apk(get_actual(0),
    recommend(0))

0.16666666666666666

In [35]:
def apk_i(user_i: int, k: int=10):
    """Get Average Precision at k for ith user."""

    if k is None:
        k = len(predicted)

    return apk(get_actual(user_i),
               recommend(user_i),
               k)

In [36]:
apk_i(0)

0.16666666666666666

In [37]:
def apk_n(n: int=len(users), k: int=10):
    """Get Average Precision at k for n users.
    
    Note that some time *after* the progress counter reaches 100% is needed for function to return a result; that's an unfortunate side-effect of multi-processing.
    
    :param n: How many users to account. Use smaller numbers for debugging and leave it at default to get the final score.
    """
    
    with concurrent.futures.ProcessPoolExecutor() as e:
        return e.map(partial(apk_i, k=k),
                     tqdm(range(n)))

In [38]:
tuple(apk_n(100))

100%|██████████| 100/100 [00:00<00:00, 1660.86it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [39]:
def mapk_n(n: int=len(users), k: int=10):
    """Get mean Average Precision at k for n users.
    
    :param n: How many users to account. Use smaller numbers for debugging and leave it at default to get the final score.
    """
    
    return np.mean(tuple(apk_n(n, k)))

In [41]:
mapk_n(10)

100%|██████████| 10/10 [00:00<00:00, 266.25it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()